In [ ]:
%reload_ext autoreload
%autoreload 2

import json
import dotenv
dotenv.load_dotenv()

from tiktokgen.agent_pipeline import agent_pipeline

from ai21 import AI21Client
from ai21.models.chat import ChatMessage

a21client = AI21Client()

In [ ]:
def generate_video_script(descriptions, event_description):
  messages = [
    ChatMessage(content="""
      You are a helpful agent that can generate a video script for a potentially viral TikTok video.
      You will be presented with a description of the event and a description of the videos and images taken on the event.
      You need to generate a script for the video.
      The format of the script should be as follows:
      [
        {
          "prompt": "description of the scene",
          "text": "script text"
        },
        ...
      ]
    """, role="system"),
    ChatMessage(content=f"""
      Video descriptions:
      {descriptions}
      
      Event description:
      {event_description}
    """, role="user")
  ]
  response = a21client.chat.completions.create(
  messages=messages,
  model="jamba-1.5-large",
  stream=True
)

  final_response = ""
  for chunk in response:
    cur_response = chunk.choices[0].delta.content
    if cur_response is not None:
      final_response += cur_response

  return final_response


In [ ]:
with open("data/local_media/video_annotations.json", "r") as file:
    annotations = json.loads(file.read())
video_descriptions = [annotations[p].replace("\n", " ") for p in annotations.keys()]
full_video_description = "\n".join([f"{i}. {desc}" for i, desc in enumerate(video_descriptions)])

event_description = "Today we have participated in the AGI house hackathon. We have built a TikTok video generator that can create videos with the same quality as the ones from TikTok influencers."

SCRIPT = generate_video_script(video_descriptions, event_description)

In [ ]:
agent_pipeline(SCRIPT, "data/output.mp4", style='Local Videos')